To answer the question, we’ll use our tool—our weapon, of course, is Python. In this case, we need GEE (Google Earth Engine) and Geemap. 

# Preparation

**GEE Key Concepts :**
GEE is like watching Netflix—it's streaming. You can choose which episode or chapter from any movie without downloading, we no need to worry about storage or switching websites in your browser. We have access to an enormous amount of data. The key concepts are:

1. Image: The fundamental raster data type in Earth Engine. *an image*
2. ImageCollection: A stack or time series of images. . *Collection of image*
3. Geometry:  The fundamental vector data type in Earth Engine. *u can call it lines*
4. Feature : A geometry with attributes. *Line with attributes*
5. FeatureCollection: A set of features. *A bunch of lines with attributes*

## Import Libraries

In [35]:
import ee
import geemap
import pandas as pd

ee.Authenticate()
ee.Initialize(project='ee-itb-kbu')

No such comm: 5622feb512004ffc8ca9598870457904
No such comm: b005e8008d5f46d2a350d1a8cd565c27
No such comm: 90da75dd364f42b0a4d3a3891c74ce70
No such comm: 04a4e43f807b41bfa2b7bd8f12d6dad5
No such comm: c13628e666c04882a7ad7ca0cb6586b9
No such comm: 922cae1daee641c583e93ce64725aa49
No such comm: ae85c0142e434863a46e17b12c46bddd
No such comm: 73b07d889c744074935708bea2c1f67a
No such comm: db8f259621084830945b92a640ee607c
No such comm: a9dc08942ff54d8e94b2b4836246d297
No such comm: 66eb7030dfa349a39f3de13e088f60e2
No such comm: 3920467298cf4d8081b556ce61298c35


## Set Area of Interest

Now, let’s set the Area of Interest. You can get the data from other sources, like the ArcGIS map server from KLHK, or simply draw your own boundaries in ArcGIS or on a site like geojson.io.

In [4]:
aoi_kbs = geemap.geojson_to_ee("/Users/yudhapratama/Documents/APAC/minidemo/Data/KBS.geojson")
aoi = geemap.geojson_to_ee("/Users/yudhapratama/Documents/APAC/minidemo/Data/KBU.geojson")

## Functions

Run code cell below. This code is based on Google Earth Engine documentation, with a small modification to standardize band names across Landsat 9 through Landsat 5 for consistency.

In [5]:
def apply_scale_factorsOLI(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )
def apply_scale_factorsOTM(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )
def maskL5sr(image):
    # Dapatkan bitmask QA
    qa = image.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 5).Or(qa.bitwiseAnd(1 << 3))
    shadow = qa.bitwiseAnd(1 << 4)
    mask = cloud.Or(shadow)
    return image.updateMask(mask.Not())
def maskC1(image):
    # Dapatkan bitmask QA
    qa = image.select('BQA')
    cloud = qa.bitwiseAnd(1 << 5).Or(qa.bitwiseAnd(1 << 3))
    shadow = qa.bitwiseAnd(1 << 4)
    mask = cloud.Or(shadow)
    return image.updateMask(mask.Not())
def renameBandsOLI(image):
    bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2']
    return image.select(bands).rename(new_bands)
    
def renameBandsOTM(image):
    bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2',]
    return image.select(bands).rename(new_bands)
def getCompositeMedian(start_year, end_year):
    start_date = f'{start_year}-01-01'
    end_date = f'{end_year}-12-31'
    
    l9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .map(maskL5sr).map(apply_scale_factorsOLI).map(renameBandsOLI))
    l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .map(maskL5sr).map(apply_scale_factorsOLI).map(renameBandsOLI))
    l7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .map(maskL5sr).map(apply_scale_factorsOTM).map(renameBandsOTM))
    l5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .map(maskL5sr).map(apply_scale_factorsOTM).map(renameBandsOTM))
    l4 = (ee.ImageCollection('LANDSAT/LT04/C02/T1_L2')
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .map(maskL5sr).map(apply_scale_factorsOTM).map(renameBandsOTM))
    combined = l8.merge(l7).merge(l5).merge(l9).merge(l4).sort('date')
    compositeMedian = combined.median().clip(aoi)
    
    return compositeMedian

# Geemap Introduction

Now, Here's how to use Geemap. Initialize the map object with geemap.Map() and display it by calling "m". The BaseMap will appear


In [36]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Basic Map

In [37]:
image = ee.Image("USGS/SRTMGL1_003").clip(aoi)
image

Name,Description
elevation,Elevation


No such comm: 6c0c150661c44d7c83ab7f2e7c5b637e
No such comm: 8023f383c1d1405f861a33f9a93c1373
No such comm: 5e2e1485b0d248e1b87caac07f230ddd
No such comm: d4942248d2574ab2b95789edad090dfd
No such comm: 655b03d7d911423280ead65d7111c474


We’ll tweak the code a little, move the center, add params and done, we have an elevation image of the North Bandung Zone, a.k.a. KBU.



In [23]:
m = geemap.Map(center=[-6.814744, 107.609810], zoom=12)
vis_params = {
    "min": 0,
    "max": 6000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],  # 'terrain'
}
m.add_layer(image, vis_params, "SRTM")
m

Map(center=[-6.814744, 107.60981], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

# Visualizing

## Timelapse

Now, let’s move on to our case. we need to understand how changes are, which means we’ll need a lot of images. With Gee, we have an access to plenty of image and geemap make things easier and faster. Now are trying to is creating a timelapse.
We’re going to look at the dynamics from 2000 to 2023.


In [25]:
timelapse = geemap.landsat_timelapse(
    aoi,
    out_gif="kbuTimelapseRedGreenBlue.gif",
    start_year=2000,
    end_year=2023,
    start_date="01-01",
    end_date="12-31",
    bands=["Red", "Green", "Blue"] ,
    frames_per_second=2,
    title="Landsat Timelapse",
    progress_bar_color="blue",
    mp4=True,
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/yudhapratama/Documents/APAC/minidemo/Notebook/kbuTimelapseRedGreenBlue.gif


Output()

Here, we can see that the brown areas keep expanding. We don’t see enough detail. We want to know exactly what the land cover looks like—whether it’s forest, buildings, or something else.

That’s why we use machine learning to predict and classify land cover.** I’ve already prepared the dataset for this part.

# Split Map

We’ll skip the land cover classification part for now, and here we are at the step where we can clearly see that we’ve successfully classified the land cover. Now, we can easily distinguish the different types of land cover.


In [26]:
start_year = 2020
end_year = 2024
compositeMedian = getCompositeMedian(start_year, end_year)
compositeMedian

In [27]:
classified = ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2021_2024').select('classification')
classified

In [28]:
m = geemap.Map(center=[-6.814744, 107.609810], zoom=12)

vis_params = {
        'bands': ['R', 'G', 'B'], 
        'min': 0,
        'max': 0.3
    }

class_params = {
    'min': 0,
    'max': 7,
    'palette': [
        '006400',  # Forest
        'ffbb22',  # Shrubland
        'ffff4c',  # Grassland
        'f096ff',  # Cropland
        'fa0000',  # Built-up
        'b4b4b4',  # Bare / Sparse vegetation
        '0064c8',  # Permanent water bodies
        '0096a0',  # Herbaceous wetland
    ]
}


left_layer = geemap.ee_tile_layer(compositeMedian,vis_params, "Classified 2024")
right_layer = geemap.ee_tile_layer(classified,class_params, "Classified 2024")

m.split_map(left_layer, right_layer)
m

Map(center=[-6.814744, 107.60981], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Time Slider

It’s time to enjoy ourselves by watching the changes unfold over time, using the images we’ve generated, rather than directly from the raw Landsat imagery

In [29]:
images = [
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_1989_1992'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_1993_1996'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_1997_2000'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2001_2004'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2005_2008'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2009_2012'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2013_2016'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2017_2020'),
    ee.Image('projects/ee-itb-kbu/assets/4yrKBU/classified_2021_2024'),
]
image_collection = ee.ImageCollection(images)

years = [1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, 2024]

def add_time(image, year):
    timestamp = ee.Date.fromYMD(year, 1, 1).millis()
    return image.set('system:time_start', timestamp)

images_with_time = []
for i, year in enumerate(years):
    image = image_collection.toList(image_collection.size()).get(i)
    image = ee.Image(image)
    images_with_time.append(add_time(image, year))

collection_with_time = ee.ImageCollection(images_with_time)

In [30]:
m = geemap.Map(center=[-6.814744, 107.609810], zoom=12)
collection_with_time = ee.ImageCollection(images_with_time)

vis_params = {
    'min': 0,
    'max': 7,
    'palette': [
        '006400',  # Forest
        'ffbb22',  # Shrubland
        'ffff4c',  # Grassland
        'f096ff',  # Cropland
        'fa0000',  # Built-up
        'b4b4b4',  # Bare / Sparse vegetation
        '0064c8',  # Permanent water bodies
        '0096a0',  # Herbaceous wetland
    ]
}

m.add_time_slider(collection_with_time, vis_params, time_interval=2)
m

Map(center=[-6.814744, 107.60981], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

No such comm: f4ab0a8977b640be81f56a177f76785a
No such comm: d2541e20cf4c4689a1c9b3835890035b
No such comm: 266950188a534a068f8facc573cebd09
No such comm: b305cc10671940dfa14dfb9b17ae230b
No such comm: cef55e151af44958a02fdeb28c03736a
No such comm: 8ea6683dd6c145409a30f817a8163c3c
No such comm: 4315d57a56b04fec882639b54ddb5ee3
No such comm: b9908d2e688b48eab95c42afe242d3b5
No such comm: 1e964f1e123145238bc3697a8927d0e4
No such comm: b72cd53e14bc457d86a3b85bf4cb5e95
No such comm: fbb69cc9db3d41e8a3f2197e34b869f0
No such comm: df8bb85ffb604a8daacc3e8501aff0c1
No such comm: e0d78caa383a47ec8db65dc88ddf6a20
No such comm: a71edfa90f22420f99aa3730a22da0da
No such comm: af4ffac3a75946a682cebe8ca42ce9aa
No such comm: 7f5ce0cc8f6947f9af28f418f2db2a01
No such comm: 1ca013547c39442483d526d77f3f6d4e
No such comm: af17ed33662f4cfb8973373469e95a10
No such comm: 74c4c70663f24135afd634b96406a283
No such comm: af3c6e2952d8414fac0e984764da01dc
No such comm: e3121610d714410babda3ff9543416fa
No such comm:

Now, we’ve seen the visual evidence in front of our eyes—KBU's green areas are shrinking significantly. We also observed earlier that from 2016 onward, the situation hasn’t improved;It looks like it's gotten worse. 


But thats it? Of course not.
We extracted the data from our land cover classification. By multiplying the land cover with emission factors from the FREL document, we created a dataset containing carbon stock data. Now, we’ll conduct statistical analysis on this dataset. We will measure the effect of the policy


## Extract data & Calculate

### Extract data from Classified Images

In [15]:
kbu_fc = ee.FeatureCollection("projects/ee-itb-kbu/assets/featureCollection/desaKBU")
landcover_images = ee.ImageCollection([
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_2021_2024").set('year_range', '2021_2024'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_2017_2020").set('year_range', '2017_2020'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_2013_2016").set('year_range', '2013_2016'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_2009_2012").set('year_range', '2009_2012'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_2005_2008").set('year_range', '2005_2008'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_2001_2004").set('year_range', '2001_2004'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_1997_2000").set('year_range', '1997_2000'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_1993_1996").set('year_range', '1993_1996'),
    ee.Image("projects/ee-itb-kbu/assets/4yrKBU/classified_1989_1992").set('year_range', '1989_1992'),
])
landcover_images


In [16]:
def calculate_area(feature):
    def calculate_area_per_image(image):
        landcover_stats = image.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=feature.geometry(),
            scale=30, 
            maxPixels=1e9
        )
        landcover_dict = ee.Dictionary(landcover_stats.get('classification'))
        area_dict = landcover_dict.map(lambda k, v: ee.Number(v).multiply(30).multiply(30).divide(1e4))
        return feature.set(area_dict).set('year_range', image.get('year_range'))
    
    return landcover_images.map(calculate_area_per_image)
desa_with_area = kbu_fc.map(calculate_area).flatten()
desa_with_area_list = desa_with_area.getInfo()['features']
results = []

for feature in desa_with_area_list:
    properties = feature['properties']
    properties['desa'] = feature['id']
    properties['year_range'] = properties.get('year_range', 'Unknown')  

In [17]:
df_results = pd.DataFrame(results)
df_results = df_results[['NAMOBJ', 'WADMKC', 'year_range', '0', '1', '2', '3', '4', '5', '7']]
df_results = df_results.rename(columns={
    'NAMOBJ': 'village',
    'WADMKC': 'district',
    '0': 'treeCover',
    '1': 'schrubland',
    '2': 'grassland',
    '3': 'cropland',
    '4': 'builtUp',
    '5': 'bare',
    '7': 'water',
})
df_results

,village,district,year_range,treeCover,schrubland,grassland,cropland,builtUp,bare,water
0,Cipanjalu,Cilengkrang,2021_2024,2.119412,0.465176,0.774000,0.488118,NaN,0.002118,0.022588
1,Cipanjalu,Cilengkrang,2017_2020,3.031412,0.114706,0.543176,0.181765,NaN,NaN,0.000353
2,Cipanjalu,Cilengkrang,2013_2016,2.693647,0.324706,0.754941,0.098118,NaN,NaN,NaN
3,Cipanjalu,Cilengkrang,2009_2012,2.752588,0.190941,0.841765,0.086118,NaN,NaN,NaN
4,Cipanjalu,Cilengkrang,2005_2008,3.579882,NaN,0.287647,0.003882,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1003,Cupunagara,Cisalak,2005_2008,0.180000,NaN,NaN,NaN,NaN,NaN,NaN
1004,Cupunagara,Cisalak,2001_2004,0.180000,NaN,NaN,NaN,NaN,NaN,NaN
1005,Cupunagara,Cisalak,1997_2000,0.180000,NaN,NaN,NaN,NaN,NaN,NaN
1006,Cupunagara,Cisalak,1993_1996,0.180000,NaN,NaN,NaN,NaN,NaN,NaN


### Calculate Carbon Stock

In [ ]:
# Calculate Carbon Stock
df_results = df_results.fillna(0)

df_results['all_area'] = (
    df_results['treeCover'] + df_results['schrubland'] + 
    df_results['grassland'] + df_results['cropland'] + 
    df_results['bare'] + df_results['builtUp'] +
    df_results['water']
)

df_results['tC_treeCover'] = df_results['treeCover'] * 347.88 * 0.47
df_results['tC_schrub'] = df_results['schrubland'] * 60.39 * 0.47
df_results['tC_grass'] = df_results['grassland'] * 4.06 * 0.47
df_results['tC_crop'] = df_results['cropland'] * 48.10 * 0.47
df_results['tC_bare'] = df_results['bare'] * 2.40 * 0.47
df_results['tC_beta'] = (
    df_results['treeCover'] * 347.88 + 
    df_results['schrubland'] * 60.39 + 
    df_results['grassland'] * 4.06 + 
    df_results['cropland'] * 48.10 + 
    df_results['bare'] * 2.40
) * 0.47
df_results['village'] = df_results['village'].str.upper()
df_results['village'] = df_results['village'].str.upper()
df_results

# Econometrics - The Effect

Now, We have the evidence. Next, We'll measure how much the policy’s impact using carbon stock. 


Principles
There are two fundamental concepts in analyzing the impact of policy. Of course, this is just the tip of an iceberg. 

First, correlation is not causation. 
For example, if that picture, we see that using Internet Explorer increase percentage of murder. So,If we stop using Internet Explorer, we’re saving lives“?. 

Second, we need to make sure that this is the only cause. In other words, We must confirm that any changes we see are due to the policy.


There are many techniques, but in this session we use Difference-in-Differences (DiD) it more intuitive.

In [21]:
data = pd.read_csv('/Users/yudhapratama/Documents/APAC/minidemo/Data/carbonStock_pyCon.csv')
data = data.set_index(['kode','periode'])
data


year_range   builtUp  tC_treeCover  tC_schrub  \
kode                  periode                                                 
CILENGKRANG_CIPANJALU 9        2021_2024  0.000000    346.762282  13.203243   
                      8        2017_2020  0.000000    495.877565   3.255731   
                      7        2013_2016  0.000000    440.651820   9.216224   
                      6        2009_2012  0.000000    450.288914   5.419541   
                      5        2005_2008  0.000000    585.323652   0.000000   
...                                  ...       ...           ...        ...   
SOREANG_SOREANG       5        2005_2008  0.058941     29.546062   0.000000   
                      4        2001_2004  0.000000     58.803589   0.000000   
                      3        1997_2000  0.000000    130.648994   0.000000   
                      2        1993_1996  0.000000    132.437916   0.000000   
                      1        1989_1992  0.000000    145.941390   0.000000   

                                tC_bare  tC_grass    tC_crop  treatment  \
kode                  periode                                             
CILENGKRANG_CIPANJALU 9        0.002389  1.476947  11.034876          1   
                      8        0.000000  1.036489   4.109155          1   
                      7        0.000000  1.440579   2.218146          1   
                      6        0.000000  1.606255   1.946862          1   
                      5        0.000000  0.548888   0.087768          1   
...                                 ...       ...        ...        ...   
SOREANG_SOREANG       5        0.096344  0.502418   8.529488          0   
                      4        0.175570  0.126615   8.681088          0   
                      3        0.124611  0.018184   1.053220          0   
                      2        0.000000  0.127962   2.002714          0   
                      1        0.000000  0.000000   1.651641          0   

                               after         w  treated  
kode                  periode                            
CILENGKRANG_CIPANJALU 9            1  1.000000        1  
                      8            1  1.000000        1  
                      7            0  1.000000        0  
                      6            0  1.000000        0  
                      5            0  1.000000        0  
...                              ...       ...      ...  
SOREANG_SOREANG       5            0  0.656488        0  
                      4            0  0.656488        0  
                      3            0  0.656488        0  
                      2            0  0.656488        0  
                      1            0  0.656488        0  

[1548 rows x 11 columns]

To estimate causal effects, most researchers typically use Stata or R because they have been popular for a long time and offer specialized tools like TWANG and MATCHIT. However, more people are now starting to use Python. The good  thing is that you can run R in  your Python project using rpy2, allowing you to use both needed! For this analysis, we'll use a basic model from the linear model called PanelOLS.


In [23]:
from linearmodels.panel import PanelOLS

mod = PanelOLS.from_formula('''tC_treeCover ~ 
treated + EntityEffects + TimeEffects''',data)

clfe = mod.fit(cov_type = 'clustered',
cluster_entity = True)
print(clfe)

                          PanelOLS Estimation Summary                           
Dep. Variable:           tC_treeCover   R-squared:                        0.0038
Estimator:                   PanelOLS   R-squared (Between):             -0.0137
No. Observations:                1548   R-squared (Within):               0.0252
Date:                Thu, Oct 24 2024   R-squared (Overall):             -0.0100
Time:                        20:40:44   Log-likelihood                -1.783e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      5.2588
Entities:                         165   P-value                           0.0220
Avg Obs:                       9.3818   Distribution:                  F(1,1374)
Min Obs:                       9.0000                                           
Max Obs:                       18.000   F-statistic (robust):             9.9172
                            

Now, let's read the regression table. We'll keep it simple and focus on the key points:

- Parameter = -7288.9: This means the treatment caused a decrease in carbon stock by 7288 tons of carbon (tC).
- Std. Error: Useful for estimating the confidence interval of the coefficient.
- R²: The closer to 1, the better. In causal effect analysis, we don’t focus too much on R² but rather on changes in the coefficient of interest.
- Two-tail p-values: If the p-value is less than 0.05, we reject the null hypothesis (H₀). In this case, the treatment is significant at a 95% confidence level.

Yes, our model is far from perfect, but I hope the main point comes across.

So far, these are our main findings, and there’s still a lot of potential for further Research

